# 1. Install Dependencies and Setup

In [ ]:
import sys
print(sys.version)
!pip --version
import numpy as np
print(np.__version__)

In [ ]:
!pip install "tensorflow<2.11" opencv-python matplotlib

In [ ]:
!pip list

In [3]:
import tensorflow as tf
import os

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth LIMIT VRAM USE
gpus = tf.config.experimental.list_physical_devices('GPU') #puts all gpus in GPU, replace 'GPU' with 'CPU' to see CPUs
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version:", tf.sysconfig.get_build_info()["cudnn_version"])

# 2. Remove dodgy images

In [6]:
import cv2
import imghdr

In [7]:
data_dir = 'data' 

In [8]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

#list stuff in data_dir
os.listdir(data_dir)

#list stuff in happy
os.listdir(os.path.join(data_dir, 'happy'))

In [9]:
for image_class in os.listdir(data_dir): #image_class is a loop variable
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [10]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#building data pipeline
data = tf.keras.utils.image_dataset_from_directory('data')

In [12]:
#loop through data
data_iterator = data.as_numpy_iterator()

In [13]:
#grabbing a batch of data, size 2: images as numpy arrays, and labels (happy or sad)
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [15]:
#normalize values to 0-1
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [17]:
train_size = int(len(data)*.7) #70% of data
val_size = int(len(data)*.2)   #...
test_size = int(len(data)*.1)

In [ ]:
train_size

In [19]:
#data already shuffled
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning Model

In [ ]:
train

In [29]:
#from tensorflow.keras.models import Model #Parallel
from tensorflow.keras.models import Sequential #single input single output
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [30]:
model = Sequential()

In [31]:
#layers (matrices)
#gives us if image is 0 or 1 (happy or sad)

#first layer, convolution, 16 filters 3x3 pixels, moves 1 pixel at a time,
#relu only keeps positive values, is an input
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
#gives max value out of 2x2 region (reduces data by 0.5)
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

#nb of filters form the nb of channels, we want all inputs -> flatten (multiplies all channels together)
#see below 30*30*16 = 14400
model.add(Flatten())
#dense = connected layers
model.add(Dense(256, activation='relu'))
#gives 0 or 1, sigmoid maps stuff to 0 and 1
model.add(Dense(1, activation='sigmoid'))

In [32]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Train

In [34]:
logdir='logs'

In [35]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
with tf.device('/GPU:0'):
    hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [42]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [43]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X) # y^
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Test

In [55]:
import cv2

In [ ]:
img = cv2.imread('happy-people.jpg')
#to see in rgb: plt.imshow(cv2.cvtColor(img, cb2.COLOR_BGR2RGB))
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

# 11. Save the Model

In [62]:
from tensorflow.keras.models import load_model

In [64]:
#to save
model.save(os.path.join('models','imageclassifier.h5'))

In [46]:
#to load
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))